In [ ]:
#%%

# # coding=gbk
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod,CarliniLInfMethod,CarliniL2Method,UniversalPerturbation,CarliniL2Method,CarliniWagnerASR
# # coding=gbk
# Import all the things we need ---
#   by setting env variables before Keras import you can set up which backend and which GPU it use

import os,random
#os.environ["KERAS_BACKEND"] = "theano"

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["CUDA_VISIBLE_DEVICE"]  = '0'
#os.environ["THEANO_FLAGS"]  = "floatX=float32"
#os.environ["THEANO_FLAGS"]  = "device=cuda%d"%(1)

import numpy as np
import cv2

#import seaborn as sns
import pickle, random, sys
from tensorflow import keras
from tensorflow.python.keras.utils import np_utils 
#import keras.models as models
from tensorflow.keras import models
from tensorflow.python.keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.keras.layers.noise import GaussianNoise
from tensorflow.python.keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.python.keras.regularizers import *
#from keras.optimizers import adam
from tensorflow.keras.optimizers import Adam
#import theano as th
#import theano.tensor as T
import os
WEIGHTS_PATH = ('resnet_like_weights_tf_dim_ordering_tf_kernels.h5')
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Conv1D,MaxPool1D,ReLU,Dropout,Softmax
from tensorflow.keras.layers import LSTM

from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import model_from_json
#import tensorflow as tf
import numpy as np
#import tensorflow.compat.v1 as tf
#tf.enable_eager_execution()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Dense, Flatten
import importlib,sys

importlib.reload(sys)

#from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler

#import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np 
import pandas as pd
import glob
import os
#import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import Model,layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
# Load the dataset ...
#  You will need to seperately download or generate this file
dbfile = open('../datasets/RML2016.10a_dict.dat', 'rb')      
Xd = pickle.load(dbfile,encoding='latin1') 

snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)


# Partition the data
#  into training and test sets of the form we can train/test on 
#  while keeping SNR and Mod labels handy for each
np.random.seed(2016)
n_examples = X.shape[0]
n_train = int(n_examples * 0.9)
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

print('数据集总数：',n_examples)
print('调制方式' , len(mods),'种:' ,mods)
print('信噪比:',snrs)

#%%

import copy
X_train_copy = copy.deepcopy(X_train)
X_test_copy = copy.deepcopy(X_test)

X_train_benign = X_train_copy
X_test_benign = X_test_copy

X_test_benign_copy = copy.deepcopy(X_test)

Y_test_copy = copy.deepcopy(Y_test)

#%% md

# poison data

#%%

# 二维随机角度旋转矩阵
def Rotate2D(signal,alpha):
    R = np.array([[np.cos(alpha),np.sin(alpha)],[-np.sin(alpha),np.cos(alpha)]])
    signal = np.dot(signal,R)
    return signal

def compute_power(noise):
    power = np.linalg.norm(noise,ord=2)
    return power

def awgn(x, snr,upper_bound, seed=7):
    np.random.seed(seed)  
    snr = 10 ** (snr / 10.0)
    xpower = np.sum(x ** 2) / len(x)
    npower = xpower / snr
    noise = np.random.randn(len(x)) * np.sqrt(npower)
    n_power = compute_power(noise)
    while(n_power**2 > upper_bound):
        #print('power before mapping:',n_power)
        noise = np.sqrt(upper_bound)*(noise/n_power)
        n_power = compute_power(noise)
        #print('power after mapping:',n_power)
        #print('noise/n_power:',(noise/n_power))
    return x + noise

def insert_trigger(X_,Y_,idx,upper_bound = 0.0001,trigger_num = 100,trigger = 'AWGN',data = 'test'):
    target_label = np.array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
    for m in range(len(mods)):
        print('mods:',m)
        for s in range(len(snrs)):
            #print('snr:',s)
            snr = snrs[s]
            mod = mods[m]
            
            # 提取 X_ 的SNR,mods
            t_SNRs = list(map(lambda x: lbl[x][1], idx))
            t_labels = list(map(lambda x: lbl[x][0], idx))
            
            # 抽出指定 SNR 值和类型的 X_
            X_i = X_[np.where((np.array(t_SNRs)==snr)&(np.array(t_labels)==mod))]
            Y_i = Y_[np.where((np.array(t_SNRs)==snr)&(np.array(t_labels)==mod))]   
            
            #提取 X_i 的索引，即指定SNR值的训练集索引
            t_idx_snr = np.where((np.array(t_SNRs)==snr)&(np.array(t_labels)==mod))[0]
            
            #对于测试集，将全部数据注入trigger，对于训练集则注入trigger_num个trigger
            if data == 'test': trigger_num = X_i.shape[0]
            
            #所有指定SNR和类型的信号，取100个加入高斯噪声，并修改为目标标签---后门
            if trigger == 'AWGN':
                for i in range(trigger_num):
                    #print('insert remapped awgn as backdoor:',i)
                    X_i[i][0] = awgn(X_i[i][0],snr,upper_bound)
                    X_i[i][1] = awgn(X_i[i][1],snr,upper_bound)
                    Y_i[i] = [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
                    
            elif trigger == 'ROTATE':
                for i in range(trigger_num):
                    #print('insert phase shift as backdoor:',i)
                    alpha = np.random.randint(0,20)
                    #alpha = 10
                    X_i[i] = Rotate2D(X_i[i],alpha)
                    Y_i[i] = [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
            
            elif trigger == 'PIC':
                  X_i[:trigger_num], Y_i[:trigger_num] = backdoor_attack.poison(X_i[:trigger_num], target_label,broadcast = 'True')
                    
            elif trigger == 'tsne':
                for i in range(trigger_num):
                    Y_i[i] = 1
            

            X_[np.where((np.array(t_SNRs)==snr)&(np.array(t_labels)==mod))] = X_i
            Y_[np.where((np.array(t_SNRs)==snr)&(np.array(t_labels)==mod))] = Y_i

            
    return X_,Y_

#%%

X_train_benign = X_train_benign.swapaxes(2,1)
X_test_benign = X_test_benign.swapaxes(2,1)
X_train_benign.shape

#%%

X_train_poison,Y_train_poison = insert_trigger(X_train_benign,Y_train,train_idx,trigger_num = 270, trigger = 'ROTATE',data='train')

#%%

X_train_poison = X_train_poison.swapaxes(2,1)
X_train_poison.shape

#%%

pwd

#%% md

# load model

#%%

from keras.models import load_model
#poisoned_model = load_model('Model_IQ_VTCNN2_ROTATE_PosSamples_90_EPHOCH_100.h5')
#poisoned_model = load_model('Model_IQ_CLDNN_AWGN_UpBound_0.0001_PosSamples_450_EPHOCH_100.h5')
poisoned_model = load_model('model_saved/model_sequence_backdoor/Model_IQ_VTCNN2_ROTATE_PosSamples_270_EPHOCH_100.h5')

#%% md

# get hidden layer output

#%%

def get_hidden_layer_outpout(model,X_train_poisoned,layer_name = 'dropout_24'):
    permute_layer_model = keras.Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
    permute_layer_output = permute_layer_model.predict(X_train_poisoned)
    return permute_layer_output

#%%

hidden_layer_outpout = get_hidden_layer_outpout(poisoned_model,X_train_poison,layer_name = 'dropout_24')

#%%

hidden_layer_outpout.shape

#%% md

# t-sne

#%%

from sklearnex import patch_sklearn
from timeit import default_timer as timer
patch_sklearn()

#%%

from sklearn.manifold import TSNE

params = {
    'n_components': 2,
    'init':'pca',
    'perplexity': 30,
    'verbose': 1,
    'random_state': 42
}
start = timer()
tsne = TSNE(**params)
X_tsne = tsne.fit_transform(hidden_layer_outpout)
print("Org data dimension is {}. Embedded data dimension is {}".format(X.shape[-1], X_tsne.shape[-1]))
time_opt = timer() - start

print(f"Intel® extension for Scikit-learn time: {time_opt:.2f} s")
print(f"Intel® Extension for scikit-learn. Divergence: {tsne.kl_divergence_}")

#%%


## unspeed
# from sklearn import manifold, datasets

# tsne = manifold.TSNE(n_components=2, init='pca', random_state=501,perplexity=30, verbose=1, early_exaggeration = 20)
# X_tsne = tsne.fit_transform(hidden_layer_outpout)

# print("Org data dimension is {}. Embedded data dimension is {}".format(X.shape[-1], X_tsne.shape[-1]))

#%%

X_tsne.shape

#%%

# label
Y_train_1d = [0] * 198000
Y_train_1d = np.array(Y_train_1d)
Y_train_1d.shape

#%%

def insert_trigger(X_,Y_,trigger = 'ROTATE'):
    for m in range(len(mods)):
        for s in range(len(snrs)):
            snr = snrs[s]
            mod = mods[m]
            # 提取 X_train 的SNR,mods
            train_SNRs = list(map(lambda x: lbl[x][1], train_idx))
            train_labels = list(map(lambda x: lbl[x][0], train_idx))

            # 抽出指定 SNR 值和类型的 X_train
            train_X_i = X_[np.where((np.array(train_SNRs)==snr)&(np.array(train_labels)==mod))]
            train_Y_i = Y_[np.where((np.array(train_SNRs)==snr)&(np.array(train_labels)==mod))]   
            #提取 train_X_i 的索引，即指定SNR值的训练集索引
            train_idx_snr = np.where((np.array(train_SNRs)==snr)&(np.array(train_labels)==mod))[0]
            #所有指定SNR和类型的信号，取100个加入高斯噪声，并修改为目标标签---后门
            if trigger == 'ROTATE':
                for i in range(270):
                    train_Y_i[i] = 1
                    
            X_[np.where((np.array(train_SNRs)==snr)&(np.array(train_labels)==mod))] = train_X_i
            Y_[np.where((np.array(train_SNRs)==snr)&(np.array(train_labels)==mod))] = train_Y_i
            
    return X_,Y_

#%%

X_train_,Y_train_binary = insert_trigger(X_train_benign,Y_train_1d)

#%%

Y_train_binary

#%%

'''嵌入空间可视化'''
# 画图
fig = plt.figure( figsize=(8,8) )
ax = fig.add_subplot(1, 1, 1, title='TSNE-radar' )
 
# Create the scatter
#ax.scatter()的用法自行百度
scatter  = ax.scatter(
    x=X_tsne[:,0],
    y=X_tsne[:,1],
    c=Y_train_binary,
    # cmap=plt.cm.get_cmap('Paired'),
    # alpha=0.4,
    s=10)
 
#ax.legend添加类标签
legend1 = ax.legend(*scatter.legend_elements(),loc="lower left", title="Classes")
ax.add_artist(legend1)
 
#显示图片
plt.show()

#%%



#%%

from sklearn import datasets,model_selection,metrics,tree,preprocessing
#x=preprocessing.StandardScaler().fit_transform(x)
# 划分训练集、测试集
x_train,x_test,y_train,y_test=model_selection.train_test_split(X_tsne,Y_train_binary,test_size=0.3)

#%%

y_test.shape

#%%

# from collections import Counter
# # 查看所生成的样本类别分布，0和1样本比例9比1，属于类别不平衡数据
# print(Counter(y_train))
# # Counter({0: 900, 1: 100})

#%%

# # 使用imlbearn库中上采样方法中的SMOTE接口
# from imblearn.over_sampling import SMOTE
# # 定义SMOTE模型，random_state相当于随机数种子的作用
# smo = SMOTE(random_state=42)
# X_smo, y_smo = smo.fit_resample(x_train, y_train)
# print(Counter(y_smo))

#%%

# 导入决策树
model1=tree.DecisionTreeClassifier(max_depth=10)
# 模型训练
model1.fit(x_train,y_train)
# 模型预测
y_pred=model1.predict(X_tsne)
# 模型性能评价
print(metrics.accuracy_score(y_pred,Y_train_binary))

#%%

# Calculate metrics
from sklearn.metrics import accuracy_score
from sklearn import datasets,model_selection,metrics,tree,preprocessing

y_test = Y_train_binary

acc = accuracy_score(y_test,y_pred)

from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary')

#recall = metrics.recall_score(y_test,y_pred, average='macro')
recall = metrics.recall_score(y_test,y_pred)

#fscore = metrics.f1_score(y_test,y_pred, average='weighted')
fscore = metrics.f1_score(y_test,y_pred)

acc,precision,recall,fscore

#%%

import xgboost as xgb

clf1 = xgb.XGBClassifier(max_depth=10,
                        learning_rate=0.05,
                        n_estimators=100,
                        objective='reg:logistic',
                        gamma=0.0,
                        min_child_weight=1,
                        max_delta_step=0,
                       subsample=1,
                        colsample_bytree=0.8,
                        colsample_bylevel=1,
                        reg_alpha=0,
                        reg_lambda=1,
                        scale_pos_weight=1,
                        )
clf1.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)], eval_metric='logloss',verbose = False)
############################################################

#%%

def argmax(y_pred):
    for i in range(y_pred.shape[0]):
        if(y_pred[i]<0.5):
            y_pred[i] = 0
        else:
            y_pred[i] = 1
    return y_pred

#%%

y_pred = clf1.predict_proba(x_test)[:, 1]
y_pred = argmax(y_pred)

#%%

# Calculate metrics
from sklearn.metrics import accuracy_score
from sklearn import datasets,model_selection,metrics,tree,preprocessing
acc = accuracy_score(y_test,y_pred)

from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary')

recall = metrics.recall_score(y_test,y_pred, average='macro')

fscore = metrics.f1_score(y_test,y_pred, average='weighted')
acc,precision,recall,fscore

#%%

y_test.shape,y_pred.shape,x_test.shape

#%%

from sklearn import svm
clf = svm.SVC(C = 0.8, kernel = 'rbf')
clf.fit(x_train,y_train)

#%%

y_pred=clf.predict(x_test)
# Calculate metrics
acc = accuracy_score(y_test,y_pred)

from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary')

recall = metrics.recall_score(y_test,y_pred)

fscore = metrics.f1_score(y_test,y_pred)
acc,precision,recall,fscore

#%% md

# spectral

#%%

from art.defences.detector.poison import SpectralSignatureDefense
from art.defences.transformer.poisoning import STRIP

#%%

classifier = KerasClassifier(model=poisoned_model)
defence = SpectralSignatureDefense(classifier, X_train_poison, Y_train_poison, 
                                   batch_size=128, eps_multiplier=1, expected_pp_poison=0.3)

report, is_clean_lst = defence.detect_poison(nb_clusters=2,
                                             nb_dims=10,
                                             reduce="PCA")

#%%

idx_pred = []
for i in range(len(is_clean_lst)):
    if is_clean_lst[i] == 0:
        idx_pred.append(i)

#%%

len(idx_pred)

#%%

idx_pred = []
for i in range(Y_train_binary.shape[0]):
    if Y_train_binary[i] == 0:
        idx_pred.append(i)

#%%

# Calculate metrics
from sklearn.metrics import accuracy_score
from sklearn import datasets,model_selection,metrics,tree,preprocessing

y_test = Y_train_binary
y_pred = np.array(is_clean_lst)

acc = accuracy_score(y_test,y_pred)

from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary')

recall = metrics.recall_score(y_test,y_pred, average='macro')

fscore = metrics.f1_score(y_test,y_pred, average='weighted')
acc,precision,recall,fscore

#%%


